In [ ]:
!pip install tf2onnx -q

import subprocess
result = subprocess.run(['git', 'clone', 'https://github.com/AnEntrypoint/StegaStamp-plus.git'], capture_output=True)
%cd StegaStamp-plus

exec(open('run_training.py').read())
